# TSP

## Use pandas to read in the excel file, `customer_list.xlsx` of customer addresses

In [2]:
from pprint import pprint
import pandas as pd
# replace any empty fields with empty string
customers_df = pd.read_excel('customer_list.xlsx').fillna('')

## We can look at the pandas customer data frame, `customer_df`.
### Note some missing cells, they are stored as empty strings, `''`

In [3]:
customers_df

,name,address,city,state,zip
0,Lula Café,2537 N Kedzie,Chicago,IL,
1,Pita Inn,4710 Dempster Street,Skokie,IL,
2,Big Jones,,Chicago,IL,
3,Pequod's Pizza,2207 N Clybourn,Chicago,IL,60614
4,L Woods Tap and Pine Lodge,7110 Lincoln Ave,Lincolnwood,IL,
5,Piece Pizza,1927 W North Ave,Chicago,IL,60622
6,Tapas Barcelona,,Evanston,IL,


### Let's now convert the Pandas structure to standard Python.  Here a List of Dictionaries.

In [4]:
customer_records = customers_df.to_dict('records')
pprint(customer_records)

[{'address': '2537 N Kedzie',
  'city': 'Chicago',
  'name': 'Lula Café',
  'state': 'IL',
  'zip': ''},
 {'address': '4710 Dempster Street',
  'city': 'Skokie',
  'name': 'Pita Inn',
  'state': 'IL',
  'zip': ''},
 {'address': '',
  'city': 'Chicago',
  'name': 'Big Jones',
  'state': 'IL ',
  'zip': ''},
 {'address': '2207 N Clybourn',
  'city': 'Chicago',
  'name': "Pequod's Pizza",
  'state': 'IL',
  'zip': 60614.0},
 {'address': '7110 Lincoln Ave',
  'city': 'Lincolnwood',
  'name': 'L Woods Tap and Pine Lodge',
  'state': 'IL',
  'zip': ''},
 {'address': '1927 W North Ave',
  'city': 'Chicago',
  'name': 'Piece Pizza',
  'state': 'IL',
  'zip': 60622.0},
 {'address': '',
  'city': 'Evanston',
  'name': 'Tapas Barcelona',
  'state': 'IL',
  'zip': ''}]


### Note:  Zip codes get treated like floats by default, for example 60647.0, so we convert each to a string and get rid of the trailing `.0`

In [5]:
for cust in customer_records:
    cust['zip'] = str(cust['zip']).replace('.0','')
pprint(customer_records)

[{'address': '2537 N Kedzie',
  'city': 'Chicago',
  'name': 'Lula Café',
  'state': 'IL',
  'zip': ''},
 {'address': '4710 Dempster Street',
  'city': 'Skokie',
  'name': 'Pita Inn',
  'state': 'IL',
  'zip': ''},
 {'address': '',
  'city': 'Chicago',
  'name': 'Big Jones',
  'state': 'IL ',
  'zip': ''},
 {'address': '2207 N Clybourn',
  'city': 'Chicago',
  'name': "Pequod's Pizza",
  'state': 'IL',
  'zip': '60614'},
 {'address': '7110 Lincoln Ave',
  'city': 'Lincolnwood',
  'name': 'L Woods Tap and Pine Lodge',
  'state': 'IL',
  'zip': ''},
 {'address': '1927 W North Ave',
  'city': 'Chicago',
  'name': 'Piece Pizza',
  'state': 'IL',
  'zip': '60622'},
 {'address': '',
  'city': 'Evanston',
  'name': 'Tapas Barcelona',
  'state': 'IL',
  'zip': ''}]


### We will be using a Google API to handle our addresses, give us geocoordinates, and travel times and directions.  Google will take the address as one long string, so let's replace our chopped up address with a single one we will call `full_address`.

### We will keep the name field since this Google API tends to discard the name, keeping only the address.  

In [6]:
for cust in customer_records:
    cust['full_address'] = cust['name'] + ', ' + cust['address'] + ', ' + cust['city'] + ', ' + cust['state'] + ' ' + cust['zip']
    del cust['address']
    del cust['city']
    del cust['state']
    del cust['zip']
pprint(customer_records)

[{'full_address': 'Lula Café, 2537 N Kedzie, Chicago, IL ',
  'name': 'Lula Café'},
 {'full_address': 'Pita Inn, 4710 Dempster Street, Skokie, IL ',
  'name': 'Pita Inn'},
 {'full_address': 'Big Jones, , Chicago, IL  ', 'name': 'Big Jones'},
 {'full_address': "Pequod's Pizza, 2207 N Clybourn, Chicago, IL 60614",
  'name': "Pequod's Pizza"},
 {'full_address': 'L Woods Tap and Pine Lodge, 7110 Lincoln Ave, Lincolnwood, '
                  'IL ',
  'name': 'L Woods Tap and Pine Lodge'},
 {'full_address': 'Piece Pizza, 1927 W North Ave, Chicago, IL 60622',
  'name': 'Piece Pizza'},
 {'full_address': 'Tapas Barcelona, , Evanston, IL ',
  'name': 'Tapas Barcelona'}]


### Our TSP begins at the company depot, which we define as the Harper Center. 
### Note we don't provide an address for the Harper Center, we will see if our Google API will take care of that!
### We will prepend this depot to our list of customers so it can be included in our TSP routing

In [7]:
depot = 'Harper Center, Chicago, IL'
# Let's add the depot as our first customer, we provide both the full_address and name keys
customer_records = [{'full_address': depot, 'name': 'Harper Center'}] + customer_records
pprint(customer_records)

[{'full_address': 'Harper Center, Chicago, IL', 'name': 'Harper Center'},
 {'full_address': 'Lula Café, 2537 N Kedzie, Chicago, IL ',
  'name': 'Lula Café'},
 {'full_address': 'Pita Inn, 4710 Dempster Street, Skokie, IL ',
  'name': 'Pita Inn'},
 {'full_address': 'Big Jones, , Chicago, IL  ', 'name': 'Big Jones'},
 {'full_address': "Pequod's Pizza, 2207 N Clybourn, Chicago, IL 60614",
  'name': "Pequod's Pizza"},
 {'full_address': 'L Woods Tap and Pine Lodge, 7110 Lincoln Ave, Lincolnwood, '
                  'IL ',
  'name': 'L Woods Tap and Pine Lodge'},
 {'full_address': 'Piece Pizza, 1927 W North Ave, Chicago, IL 60622',
  'name': 'Piece Pizza'},
 {'full_address': 'Tapas Barcelona, , Evanston, IL ',
  'name': 'Tapas Barcelona'}]


## We use `google-map-services-python`, the GitHub site and documentation is here: https://github.com/googlemaps/google-maps-services-python.  This is a library of python code specifically written to interace with google map services using python.
### The github site explains how to install the `googlemaps` python module, and provides links on how to get your own Google Map Services API key


In [8]:
import googlemaps
import os    # You won't need this unless you hide your Google Key like I did
# We first need to provide the googlemaps api our Google Services Key.  I have my key hidden in my OS as an environment variable. 
googlemap = googlemaps.Client(key=os.environ["GOOGLE_API_KEY"])
# You can instead just put your key in directly, so, for example if your key is 'AIXX88' then you would use the following, uncommented
#   googlemap = googlemaps.Client(key='AIX88')

ModuleNotFoundError: No module named 'googlemaps'

In [9]:
#  Let try out this API by calling the geocode method for the second customer
#  By calling googlemap.geocode('Some address'), we are making an internet call to Google's API asking google for 
#  information about an address.
geocode_result =  googlemap.geocode(customer_records[1]['full_address'])
print(f"geocode info for customer {customer_records[1]['full_address']}\n")
pprint(geocode_result)
# The result is a List, which contains one element, a Dictionary
print("\n\n")
print(f"The result is a {type(geocode_result)}")
print(f" of length {len(geocode_result)}")

NameError: name 'googlemap' is not defined

In [10]:
#  We add to each customer dictionary, `google_address` and the location as a List:  `[lat,lng]`
for cust in customer_records:
    geocode_result = googlemap.geocode(cust['full_address'])
    cust['google_address'] = geocode_result[0]['formatted_address'] 
    cust['location'] = [geocode_result[0]['geometry']['location']['lat'],geocode_result[0]['geometry']['location']['lng']]
pprint(customer_records)

NameError: name 'googlemap' is not defined

In [11]:
# Let's make a simple list, customer_addresses,  of google addresses
# This will be useful when we get to mapping
#
# Python is powerful, it can be done in one line!
# customer_addresses = [cust['google_address'] for cust in customer_records]
# But we'll go at it a bit more methodically here:
customer_addresses=[]
for cust in customer_records:
    customer_addresses.append(cust['google_address'])
pprint(customer_addresses)

KeyError: 'google_address'

## We now will call Google's Directions API:   https://developers.google.com/maps/documentation/directions/start
### This API will provide us a result, an object, that describes the time and distance between two addresses, accounting for traffic and time of travel if desired

In [12]:
from datetime import datetime

now = datetime.now()
# We ask google to consider the traffic now
# We provide first a list of Origin addresses and then Destination addresses, for us, they are both given by customer_addresses
my_dist = googlemap.distance_matrix(customer_addresses,customer_addresses,
                                            mode="driving",
                                            units="imperial",
                                            departure_time=now)

pprint(my_dist)

NameError: name 'googlemap' is not defined

### We now have in effect, a square matrix of travel times between each customer pair.
### `my_dist` is a big Dictionary, the `rows` key yields a List of Dictionaries, a Dictionary for each row(origin)
###  The Key for each of these Dictionaries that we are interested in is `elements`.
###  Each of these 'elements' is a List of Dictionaries, a dictionary for each Column(destination)
###  We can then grab out the Key `duration_in_traffic` from each of these Dictionaries, which (is anyone still with me?) is another Dictionary that finally contains the Key `value`, which gives the time in seconds

In [34]:
# travel_times will hold our square matrix of travel times
travel_times = []
# loop through the list of rows(origins)
for row in my_dist['rows']:
    # put in a new empty list for the new row
    travel_times.append([])
    # now, for each row loop through all the columns(destinations)
    for column in row['elements']:
        duration = column['duration_in_traffic']['value']
        # append this time to the last element of travel_times, which is our current list for this row
        travel_times[-1].append(duration)
pprint(travel_times)

[[3, 2246, 2754, 2494, 2137, 2572, 1938, 3153],
 [2162, 22, 1189, 1389, 790, 973, 714, 1856],
 [2875, 1439, 0, 1564, 1630, 484, 1535, 790],
 [2006, 1566, 1621, 0, 1130, 1051, 1400, 1179],
 [1775, 708, 1270, 967, 8, 1034, 350, 2000],
 [2735, 1312, 674, 1137, 1443, 0, 1363, 1207],
 [1796, 661, 1297, 1381, 482, 1066, 7, 1948],
 [3030, 1971, 652, 1142, 2131, 1074, 2045, 7]]


## Solving the TSP
### The TSP is complicated to solve to optimality.  Formulating its solution with gurobi is not straightforward.  In fact, if the problem instance gets large enough, it may be impractical to find the optimal solution.  Fortunately, most real-world problems, with hundreds or even thousands of points, can be solved.  There are also many heuristic approaches that can get close to optimal.
### Here we use a python packaged solver for simplicity.  It is described here:  https://pypi.org/project/tsp/

In [13]:
# https://pypi.org/project/tsp/ to download the module and find some limited documentation
# Correspondence with the author confirms that the module returns optimal solutions using PuLP, and instances can be
# non-symmetric.
import tsp
r = range(len(travel_times))
# Dictionary of distance.  Each key is a tuple (i,j) with value travel_times[i][j]
mydist = {}
for i in r:
    for j in r:
        mydist[(i,j)] = travel_times[i][j]
# provide the tsp solver the range of values and the dictionary of distances
tsp_raw_solution = tsp.tsp(r, mydist)

In [14]:
# The raw_solution is a tuple whose first entry is the length of the optimal tour, 
# followed by an array of the optimal route
tsp_raw_solution


(9747.0, [0, 6, 4, 1, 5, 2, 7, 3])

In [1]:
# tsp_route picks off just the optimal route array
cost, tsp_route = tsp_raw_solution

NameError: name 'tsp_raw_solution' is not defined

## Our array `tsp_route` provides the optimal TSP sequence, the return to the depot is implied but *not* explicitly placed at the end of the array

In [16]:
tsp_route

[0, 6, 4, 1, 5, 2, 7, 3]

In [17]:
# Sequence of TSP tour using our customer addresses ... return to the depot is not output but implied in route
for customer in tsp_route:
    print(customer_records[customer]['google_address'])
    print(customer_records[customer]['location'])

Chicago Booth Harper Center, 5807 S Woodlawn Ave, Chicago, IL 60637, USA
[41.7891387, -87.5954555]
1927 W North Ave, Chicago, IL 60622, USA
[41.9103001, -87.67612160000002]
2207 N Clybourn Ave, Chicago, IL 60614, USA
[41.921885, -87.6644455]
2537 N Kedzie Ave, Chicago, IL 60647, USA
[41.927684, -87.70679400000002]
7110 Lincoln Ave, Lincolnwood, IL 60712, USA
[42.01014079999999, -87.7378651]
4710 Dempster Street, Skokie, IL 60076, USA
[42.0410639, -87.7451663]
1615 Chicago Ave, Evanston, IL 60201, USA
[42.0466634, -87.6791169]
5347 N Clark St, Chicago, IL 60640, USA
[41.9794569, -87.66814749999999]


In [18]:
map_marker_locations=[]
map_marker_info_boxes=[]
sequence_num = 0
for customer in tsp_route:
    sequence_num = sequence_num + 1
    # each map_marker has a location and a popup_html
    print(customer_records[customer]['location'])
    map_marker_locations.append(customer_records[customer]['location'])
    map_marker_info_boxes.append('<b>' +  str(sequence_num) + ': ' + customer_records[customer]['name'] + '<br></b>' + customer_records[customer]['google_address'])


[41.7891387, -87.5954555]
[41.9103001, -87.67612160000002]
[41.921885, -87.6644455]
[41.927684, -87.70679400000002]
[42.01014079999999, -87.7378651]
[42.0410639, -87.7451663]
[42.0466634, -87.6791169]
[41.9794569, -87.66814749999999]


In [19]:
# Let's take a look at our map_markers to check em out
map_marker_locations

[[41.7891387, -87.5954555],
 [41.9103001, -87.67612160000002],
 [41.921885, -87.6644455],
 [41.927684, -87.70679400000002],
 [42.01014079999999, -87.7378651],
 [42.0410639, -87.7451663],
 [42.0466634, -87.6791169],
 [41.9794569, -87.66814749999999]]

In [20]:
map_marker_info_boxes

['<b>1: Harper Center<br></b>Chicago Booth Harper Center, 5807 S Woodlawn Ave, Chicago, IL 60637, USA',
 '<b>2: Piece Pizza<br></b>1927 W North Ave, Chicago, IL 60622, USA',
 "<b>3: Pequod's Pizza<br></b>2207 N Clybourn Ave, Chicago, IL 60614, USA",
 '<b>4: Lula Café<br></b>2537 N Kedzie Ave, Chicago, IL 60647, USA',
 '<b>5: L Woods Tap and Pine Lodge<br></b>7110 Lincoln Ave, Lincolnwood, IL 60712, USA',
 '<b>6: Pita Inn<br></b>4710 Dempster Street, Skokie, IL 60076, USA',
 '<b>7: Tapas Barcelona<br></b>1615 Chicago Ave, Evanston, IL 60201, USA',
 '<b>8: Big Jones<br></b>5347 N Clark St, Chicago, IL 60640, USA']

In [21]:
import gmaps 
gmaps.configure(api_key=os.environ["GOOGLE_API_KEY"])
fig = gmaps.figure()
markers = gmaps.marker_layer(map_marker_locations, info_box_content=map_marker_info_boxes)
directions_lay = gmaps.directions_layer(map_marker_locations[0],map_marker_locations[0],waypoints=map_marker_locations,show_markers=False)
fig.add_layer(directions_lay)
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))

### Homework/Classroom work 
- Suppose I don't really care about the time spent returning to the depot.  That is, I want to find the least cost route starting at the depot and visiting all customers.  How can I do this using the TSP package?
- Now suppose I want to start at the depot, visit all customers, but specify which customer I want to end my tour?  This may be useful for example when an Uber driver wants to pick up a pool of customers and end the pick ups at the customer closest to the destinations.  How would I model and solve this problem using the TSP package?